In [ ]:
using DistMesh
using Plots
using SparseArrays
using Polynomials
using SymRCM

# Functions from last week

In [ ]:
function xy(x0::Float64, y0::Float64, L1::Float64, L2::Float64, noelms1::Int64, noelms2::Int64)
    # Computes the x-coordinates and y-coordinates in a given range, with a given grid definition.
    # Author: Clara Hollenbeck
    # x0, y0: minimum x and y value
    # L1, L", maximum x and y value respectively
    # noelms1, noelms2: no. of elements in the x and y direction respectively.

    VX= repeat(collect(LinRange(x0, x0 + L1, noelms1+1)), inner = noelms2+1)
    VY = repeat(collect(LinRange(y0 + L2, y0, noelms2+1)), noelms1+1)
    return VX, VY
end

function conelmtab(noelms1::Int64, noelms2::Int64)
    novertices = noelms1 *noelms2*2
    EToV = zeros(Int64, novertices, 3)

    for i in 1: noelms1
        v1 = collect(range(1, noelms2)) .+ (noelms2+1)*(i-1)
        v2 = collect(range(noelms2 + 3, 2*noelms2+2)) .+ (noelms2+1)*(i-1)
        v3 = collect(range(noelms2 + 2, 2*noelms2 +1)) .+ (noelms2+1)*(i-1)
        v4 = collect(range(2 , noelms2+1)) .+ (noelms2+1)*(i-1)

        EToV[1 + noelms2*2 *(i-1) : noelms2*2 *i , :] = vcat(hcat(v1,v2,v3), hcat(v1, v4, v2))
    end
    return EToV[sortperm(EToV[:, 1]), :]
end

function basfun(VX, VY, EToV)
    xks = VX[EToV[:, [3,1,2]]]
    xjs = VX[EToV[:, [2,3,1]]]
    yks = VY[EToV[:, [3,1,2]]]
    yjs = VY[EToV[:, [2,3,1]]]

    as = xjs .* yks .- xks .* yjs
    bs = yjs .- yks
    cs = xks .- xjs

    deltas = sum(as, dims=2)./2
    return bs,cs, deltas
end

function constructBeds(
    VX::Vector{Float64},
    VY::Vector{Float64},
    EToV::Matrix{Int64},
    tol::Float64,
    fd::Function,
)::Matrix{Int64}
    xc = (VX[EToV] + VX[EToV[:, [2, 3, 1]]]) ./ 2
    yc = (VY[EToV] + VY[EToV[:, [2, 3, 1]]]) ./ 2
    return getindex.(findall(abs.(fd.(xc, yc)) .<= tol), [1 2])
end

function constructBnodes(
    VX::Vector{Float64},
    VY::Vector{Float64},
    tol::Float64,
    fd::Function,
)::Vector{Int64}
    return findall(abs.(fd.(VX, VY)) .<= tol)
end

# Time dependent assembly

In [ ]:
function assembly(
    VX::Vector{Float64},
    VY::Vector{Float64},
    EToV::Matrix{Int64},
    lam1::Float64,
    lam2::Float64,
    qt::Vector{Float64}
    ct:: Vector{Float64}
)::Tuple{SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Float64, Int64}, Vector{Float64}}
    N = size(EToV)[1]
    M = length(VX)

    R = spzeros(M, M)
    S = spzeros(M, M)
    B = zeros(M)

    bs, cs, deltas = basfun(VX, VY, EToV)
    qs = abs.(deltas) .* sum(qt[EToV], dims=2) / 9
    cs = ct .* abs.(deltas)./12
    C = [2 1 1; 1 2 1; 1 1 2]

    for n in 1:N
        delta = deltas[n]
        q = qs[n]
        b = bs[n, :]
        c = cs[n, :]

        for r in 1:3
            i = EToV[n,r]
            B[i] += q

            for s in r:3
                j = EToV[n,s]
                i2 = min(i, j)
                j2 = max(i, j)
                kn = (lam1*b[r]*b[s] + lam2*c[r]*c[s]) / (4 * abs(delta))
                cn = cs[n]*C[i2, j2]
                R[i2, j2] += kn
                S[i2, j2] += cn
            end
        end
    end

    return R, S, B
end

In [ ]:
function dirbc(bnodes,f,R,S,b)
    M = size(R)[1]
    d = zeros(d)

    for i in bnodes
        b[i] = 0

        idx = findall(!iszero, R[:, i])
        temp = R[idx, i]*f[i] 
        b[idx] -= temp
        d[idx] += temp

        R[idx, i] = 0
        S[idx, i] = 0
        R[i,i] = 1
    end
    return R, S, b, d
end